In [1]:
import sys
sys.path.append("../../")
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.image as mpimg
import time
import pandas as pd
from pandas.plotting import scatter_matrix
import h5py
#from tqdm.keras import TqdmCallback
from transport_nets.models.NVP import NVP

tfd = tfp.distributions
tfb = tfp.bijectors

In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
file = h5py.File('BeamSamples.h5','r')
joint_samples = np.array(file['Samples/'],dtype=np.float32)
scaler = MinMaxScaler()
scaled_samples = scaler.fit_transform(joint_samples)

NUM_SAMPLES = joint_samples.shape[0]
BATCH_SIZE = 50
dataset = tf.data.Dataset.from_tensor_slices(scaled_samples)
dataset = dataset.shuffle(buffer_size=NUM_SAMPLES).batch(BATCH_SIZE)

OUTPUT_DIM = 9
NEURON_LIST = [100,100]
NUM_BLOCKS = 8
masks = NUM_BLOCKS*[5]

nvp = NVP(output_dim=OUTPUT_DIM,
          num_layers = len(masks), 
          neuron_list = NEURON_LIST)

# Adam optimizer parameters
LR = 1e-4
beta_1 = 0.9
beta_2 = 0.999
opt = tf.keras.optimizers.Adam(learning_rate=LR, beta_1=beta_1, beta_2=beta_2, epsilon=1e-08)
nvp.compile(opt)

N_EPOCHS = 20
# Train the model with the new callback
nvp.batch_norm_mode(True)
history = nvp.fit(dataset, epochs=N_EPOCHS)  # Pass callback to training
# Set the training variables to not update when doing passes
nvp.batch_norm_mode(False)

Epoch 1/20
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
`AffineScalar` bijector is deprecated; please use `tfb.Shift(loc)(tfb.Scale(...))` instead.
1000/1000 [==============================] - 16s 16ms/step - Negative Log Likelihood: -29.5340
Epoch 2/20
1000/1000 [==============================] - 15s 15ms/step - Negative Log Likelihood: -32.5547
Epoch 3/20
1000/1000 [==============================] - 15s 15ms/step - Negative Log Likelihood: -32.9000
Epoch 4/20
1000/1000 [==============================] - 16s 16ms/step - Negative Log Likelihood: -33.1304
Epoch 5/20
 615/1000 [=================>............] - ETA: 6s - Negative Log Likelihood: -33.2383

KeyboardInterrupt: 

In [ ]:
from scipy.ndimage.filters import gaussian_filter1d
ysmoothed = gaussian_filter1d(history.history['Negative Log Likelihood'], sigma=2)
plt.plot(history.history['Negative Log Likelihood'],'b')
plt.plot(ysmoothed, color = 'r')
plt.xlabel('epochs')
plt.ylabel('negative log likelihood');

In [ ]:
plt.figure(figsize=(15,15))
scatter_matrix(pd.DataFrame(joint_samples[::100,:]))
plt.savefig('Euler-Beam-joint-density-samples.png')
plt.show();

In [ ]:
fig = plt.figure(figsize=(15,15))
x_forward = scaler.inverse_transform(nvp.sample(500).numpy())
scatter_matrix(pd.DataFrame(x_forward))
plt.savefig('Euler-Beam-joint-density-nvp.png')
plt.show();

### Algorithm 1 RTO Metropolis–Hastings (RTO-MH)

1. Find $v_{ref}$ using (3)  

$$(3)\qquad v_{ref} = arg \min_{v} \frac{1}{2} ||H(v)||^2 $$

2. Determine $\nabla H(v_{ref})$
3. Compute Q, whose columns are an orthonomal basis for the range of $\nabla H(v_{ref})$
4. for $i=1,...,n_{samps}$ do in parallel
5. &nbsp;&nbsp;&nbsp;&nbsp; Sample $\eta^{(i)}$ from an (n+m) dimensional standard normal distribution
6. &nbsp;&nbsp;&nbsp;&nbsp; Solve for a proposal sample $v_{prop}^{(i)}$ using (5)
7. &nbsp;&nbsp;&nbsp;&nbsp; Compute $w(v_{prop}^{(i)})$ from (8)

$$(5) \qquad v_{prop}^{(i)} = arg \min_{v} \frac{1}{2}\left|\left|Q^T(H(v)-\eta^{(i)})\right|\right|^{\ 2}$$

$$(8) \qquad w(v) = |det(Q^T \nabla H(v))|^{-1}exp\left(-\frac{1}{2}||H(v)||^2 + \frac{1}{2}||Q^TH(v)||^2\right)$$

8. Set $v^{(0)} = v_{ref}$
9. for $i=1,...,n_{samps}$ do in series
10. &nbsp;&nbsp;&nbsp;&nbsp; Sample $t$ from a uniform distribution on $[0,1]$
11. &nbsp;&nbsp;&nbsp;&nbsp; if $t < w(v_{prop}^{(i)})\ /\ w(v^{i-1})$ then
12. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $v^{(i)} = v_{prop}^{(i)}$
13. &nbsp;&nbsp;&nbsp;&nbsp; else
14. &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $v^{(i)} = v^{(i-1)}$

In [ ]:
from transport_nets.mcmc import RTO_MetropolisHastings
num_params = 3
m_y_scaled = scaled_samples[0,:]
y_obs = m_y_scaled[num_params:]
noise_std = 0.01
N_samples=1000
rto_mh = RTO_MetropolisHastings(num_params=num_params, y_obs=y_obs, model = nvp, noise_std = noise_std)
# time bar only for sequential sampling part
# self.ref_samples attribute created during run, which are final samples before forward pass
target_samples, acc_rate, time_total = rto_mh.run(N_samples)                                                           
X = scaler.inverse_transform(target_samples.numpy())

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(15,5))
m_y_obs_unscaled = scaler.inverse_transform(np.reshape(m_y_scaled,[1,-1]))
m1,m2,m3 = tuple(m_y_obs_unscaled[0,:num_params])

axs[0].hist(X[:,0],50,density=True)
axs[0].axvline(m1,ymax=0.95,c='r',ls='--',lw=2.0,label='true val')
axs[0].set_xlabel('m1')
axs[0].legend()

axs[1].hist(X[:,1],50,density=True)
axs[1].axvline(m2,ymax=0.95,c='r',lw=2.0,ls='--',label='true val')
axs[1].set_xlabel('m2')
axs[1].legend()

axs[2].hist(X[:,2],50,density=True)
axs[2].axvline(m3,ymax=0.95,c='r',lw=2.0,ls='--',label='true val')
axs[2].set_xlabel('m3')
axs[2].legend()

plt.suptitle('Number of Samples: %d\n Acceptance Rate: %0.3f \n Time: %0.2f s'%(N_samples,acc_rate,time_total))
plt.subplots_adjust(top=0.85)
plt.savefig('Euler_Beam_RTO_MH_samples_1e-2_obs_noise.png')